# Classic Californian Housing Dataset

## Purpouse
The Housing dataset containg information about houses sold in last 50 yeras, the main goal of this exercise is to create the ML 
model that could predict the price of the house, based on some features.
Azure AML allows user to choose huge amount of different parameters, but most interesting option is the ability to let the model choose the best regressors automaticaly. 
In this exercise I will try to perform different types of ML to see if my models are better or worst.

Two main approaches where taken, the click-click approach, where the experiment was created in AML UI, and second was created via the jupiterNotebook.

## Dataset
Dataset will be added to github repository, it is pre-prepaerd for machine learing. The aim of this exercise is not to perform the data analisys and clearing, so the used dataset will be pre-cleared.

Data set contains 25 columns, 21596 records each:

### TARGET COLUMN
- price - $

### FEATURE COLUMNS
- bedrooms - number of bedrooms (can contain 0.5 bedroom, when connected to other room)
- bathrooms - number of bathrooms (can contain 0.5 bathroom, when without shower or tub)
- sqft_living
- sqft_lot
- floors - number of floors (can contain 0.5)
- waterfront - the scale 0-10 (it represents the quality)
- view - the scale 0-10 (it represents the quality)
- condition - the scale 0-10 (it represents the quality)
- grade -subjective, the scale 0-10 (it represents the quality)
- sqft_above
- sqft_basement
- yr_built
- yr_renovated
- lat - coordinates of lattitude
- long - coordinates of longitude 
- sqft_living15 - living space of 15 closest flats/houses
- sqft_lot15 - lot area of 15 closest flats/houses
- month - month of selling
- year - year of selling 

### ADVANCED FEATURES
some additional features created to enlarge of simplify relations
- bedrooms_squared - linear relation between price is better in case of squared(bathrooms) than in just bathrooms
- bed_bath_rooms - the connected value of bedrooms*bathrooms has strong corelation with price
- log_sqft_living - the data correction to achieve the strait distribution not left-weighted
- sqft_rooms - the sum of all spaces insige house
- condition_advanced - condition*(yr_built+yr_renovated) based on the year built and year renovated we can guite well asses the real condition

## AML UI approach - click-click approach
Here anly basic steps of this approach will be described, the written approach seems to be much more flexible and overall better.

Some of the beggining steps will be later reused in the Jupiter approach.

1. Go to https://ml.azure.com - if you have no resource you will be prompted to create one, so just follow the instructions, or click on create new resource.
2. Compute Tab - New Compute Instance - the best one is the cheapest one, later it will be the **test instance**
3. Compute Tab - New Compute Cluster - again best is cheapest, later it will be used as **train instance**
4. Upload Dataset - in dataset tab - all instructions are prompted to one, as one goes
5. Check data in data set, the uppermentioned dataset is prepared, but some may need transformations
6. Explore data in Explore Tab
7. Go to AML tab - the blot with a gear icon - **New Run** - basically one is again prompted all the way
8. Select Dataset
9. Create Experiment - change time prom 3h to 1h, to save some money
10. Check the results

## Jupyter approach
Whole approach will be presented in the cells below.

### Set up SDK
First Cell updates SDK, this is an answer to the frequently showing up problem.

In [3]:
pip install --upgrade --upgrade-strategy eager azureml-sdk

Requirement already up-to-date: azureml-sdk in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (1.21.0)
Requirement already up-to-date: azureml-dataset-runtime[fuse]~=1.21.0 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-sdk) (1.21.0)
Requirement already up-to-date: azureml-pipeline~=1.21.0 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-sdk) (1.21.0)
Requirement already up-to-date: azureml-core~=1.21.0 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-sdk) (1.21.0.post1)
Requirement already up-to-date: azureml-train-automl-client~=1.21.0 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-sdk) (1.21.0)
Requirement already up-to-date: azureml-train~=1.21.0 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-sdk) (1.21.0)
Requirement already up-to-date: pyarrow<2.0.0,>=0.17.0 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-dataset-runtime[f

Despite all errors in the upper cell, the SDK seems to have installed correctly

In [1]:
import azureml.core

print(azureml.core.VERSION)

1.21.0


### Load and set-up Dataset

In [2]:
from azureml.core import Workspace, Dataset

subscription_id = '2bf6a9f2-6624-4947-9b91-d49100c13f5c'
resource_group = 'AZURE_AI'
workspace_name = 'azureml'

work_space = Workspace(subscription_id, resource_group, workspace_name)

In [3]:
# Load data into AML dataset format
housing_ds = work_space.datasets['housing']

In [4]:
housing_ds

{
  "source": [
    "('workspaceblobstore', 'UI/01-26-2021_105808_UTC/housing_adv.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "46df87cb-90f2-4bba-9d0f-f85f7e4dea0b",
    "name": "housing",
    "version": 2,
    "workspace": "Workspace.create(name='azureml', subscription_id='2bf6a9f2-6624-4947-9b91-d49100c13f5c', resource_group='AZURE_AI')"
  }
}

To perform visual operations I will use pandas dataframe, it is easier to operate. The Trainging will be performed on datasets

In [5]:
housing_df = housing_ds.to_pandas_dataframe()

In [6]:
housing_df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,long,sqft_living15,sqft_lot15,month,year,bedrooms_squared,bed_bath_rooms,log_sqft_living,sqft_rooms,condition_advanced
0,221900.0,3.0,1.00,1180.0,5650,1.0,0,0,3,7,...,-122.257,1340.0,5650.0,10,2014,9.0,3.00,7.073270,295.000000,11730
1,538000.0,3.0,2.25,2570.0,7242,2.0,0,0,3,7,...,-122.319,1690.0,7639.0,12,2014,9.0,6.75,7.851661,489.523810,11826
2,180000.0,2.0,1.00,770.0,10000,1.0,0,0,3,6,...,-122.233,2720.0,8062.0,2,2015,4.0,2.00,6.646391,256.666667,11598
3,604000.0,4.0,3.00,1960.0,5000,1.0,0,0,5,7,...,-122.393,1360.0,5000.0,12,2014,16.0,12.00,7.580700,280.000000,19650
4,510000.0,3.0,2.00,1680.0,8080,1.0,0,0,3,8,...,-122.045,1800.0,7503.0,2,2015,9.0,6.00,7.426549,336.000000,11922


In [7]:
housing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21596 entries, 0 to 21595
Data columns (total 25 columns):
price                 21596 non-null float64
bedrooms              21596 non-null float64
bathrooms             21596 non-null float64
sqft_living           21596 non-null float64
sqft_lot              21596 non-null int64
floors                21596 non-null float64
waterfront            21596 non-null int64
view                  21596 non-null int64
condition             21596 non-null int64
grade                 21596 non-null int64
sqft_above            21596 non-null int64
sqft_basement         21596 non-null int64
yr_built              21596 non-null int64
yr_renovated          21596 non-null int64
lat                   21596 non-null float64
long                  21596 non-null float64
sqft_living15         21596 non-null float64
sqft_lot15            21596 non-null float64
month                 21596 non-null int64
year                  21596 non-null int64
bedrooms_squa

One can easili notice, that data was prepared well: no null, only numericly continuous and meaningful data and the previously categorical data was transformed into separate columns or numbers.

### Train-Test split
Theoretically, we should always create 3 sets of data, training, validation and test. 
But due to the fact that here i want to learn only the Azure functionality, i will use test dataset as the validation dataset.
Additionaly, what i have learned later, the AML creates validation dataset on its own during learing process.

In [8]:
train_ds, test_ds = housing_ds.random_split(0.7)

In [9]:
train_ds.to_pandas_dataframe().describe().T

,count,mean,std,min,25%,50%,75%,max
price,15149.0,540446.091755,369067.328341,78000.000000,321027.000000,452500.00000,645000.000000,7.700000e+06
bedrooms,15149.0,3.373358,0.910081,1.000000,3.000000,3.00000,4.000000,1.100000e+01
bathrooms,15149.0,2.119645,0.773188,0.500000,1.750000,2.25000,2.500000,8.000000e+00
sqft_living,15149.0,2084.808304,927.282072,370.000000,1420.000000,1920.00000,2560.000000,1.354000e+04
sqft_lot,15149.0,15043.513037,42338.476928,520.000000,5036.000000,7616.00000,10716.000000,1.651359e+06
floors,15149.0,1.494422,0.541472,1.000000,1.000000,1.50000,2.000000,3.500000e+00
waterfront,15149.0,0.007195,0.084522,0.000000,0.000000,0.00000,0.000000,1.000000e+00
view,15149.0,0.233283,0.763125,0.000000,0.000000,0.00000,0.000000,4.000000e+00
condition,15149.0,3.402997,0.646980,1.000000,3.000000,3.00000,4.000000,5.000000e+00
grade,15149.0,7.659185,1.177068,3.000000,7.000000,7.00000,8.000000,1.300000e+01


In [10]:
test_ds.to_pandas_dataframe().describe().T

,count,mean,std,min,25%,50%,75%,max
price,6447.0,539616.040639,362604.090690,82500.000000,322734.000000,446500.000000,640000.000000,5.570000e+06
bedrooms,6447.0,3.368233,0.889995,1.000000,3.000000,3.000000,4.000000,1.000000e+01
bathrooms,6447.0,2.106910,0.759047,0.750000,1.750000,2.250000,2.500000,6.500000e+00
sqft_living,6447.0,2069.851094,896.213054,390.000000,1430.000000,1910.000000,2530.000000,9.200000e+03
sqft_lot,6447.0,15232.162556,39157.153601,600.000000,5080.000000,7620.000000,10608.500000,1.074218e+06
floors,6447.0,1.493408,0.535502,1.000000,1.000000,1.500000,2.000000,3.500000e+00
waterfront,6447.0,0.008376,0.091143,0.000000,0.000000,0.000000,0.000000,1.000000e+00
view,6447.0,0.236699,0.774116,0.000000,0.000000,0.000000,0.000000,4.000000e+00
condition,6447.0,3.425624,0.658377,1.000000,3.000000,3.000000,4.000000,5.000000e+00
grade,6447.0,7.655033,1.164209,4.000000,7.000000,7.000000,8.000000,1.300000e+01


Train-Test split was performed correctly, the statistics of both dataset are quite similar, so we can presume that data was split OK.

### Cluster - setup and use
This part of the exercise was mostly taken from one of the examples found on the aml webpage.

#### Testinstance

In [11]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

# list targets form active work_space
ComputeTarget.list(work_space)

[{
   "id": "/subscriptions/2bf6a9f2-6624-4947-9b91-d49100c13f5c/resourceGroups/AZURE_AI/providers/Microsoft.MachineLearningServices/workspaces/azureml/computes/testinstance001",
   "name": "testinstance001",
   "location": "northeurope",
   "tags": null,
   "properties": {
     "description": null,
     "computeType": "ComputeInstance",
     "computeLocation": "northeurope",
     "resourceId": null,
     "provisioningErrors": null,
     "provisioningState": "Succeeded",
     "properties": {
       "vmSize": "STANDARD_DS3_V2",
       "applications": [
         {
           "displayName": "Jupyter",
           "endpointUri": "https://testinstance001.northeurope.instances.azureml.ms"
         },
         {
           "displayName": "Jupyter Lab",
           "endpointUri": "https://testinstance001.northeurope.instances.azureml.ms/lab"
         },
         {
           "displayName": "RStudio",
           "endpointUri": "https://testinstance001-8787.northeurope.instances.azureml.ms"
      

#### Traininstance

In [12]:
# Type the name of you cluster
amlcompute_cluster_name = "traininstance002"

found = False
# Check if this compute target already exists in the workspace.

cts = work_space.compute_targets
if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'AmlCompute':
     found = True
     print('Found existing training cluster.')
     # Get existing cluster
     aml_remote_compute = cts[amlcompute_cluster_name]
    
print('Checking cluster status...')
aml_remote_compute.wait_for_completion(show_output = True, min_node_count = 0, timeout_in_minutes = 20)

Found existing training cluster.
Checking cluster status...
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [13]:
# additional details info
aml_remote_compute.get_status().serialize()

{'currentNodeCount': 1,
 'targetNodeCount': 1,
 'nodeStateCounts': {'preparingNodeCount': 0,
  'runningNodeCount': 0,
  'idleNodeCount': 1,
  'unusableNodeCount': 0,
  'leavingNodeCount': 0,
  'preemptedNodeCount': 0},
 'allocationState': 'Steady',
 'allocationStateTransitionTime': '2021-01-26T12:18:36.053000+00:00',
 'errors': None,
 'creationTime': '2021-01-26T10:36:55.007345+00:00',
 'modifiedTime': '2021-01-26T10:37:10.757331+00:00',
 'provisioningState': 'Succeeded',
 'provisioningStateTransitionTime': None,
 'scaleSettings': {'minNodeCount': 0,
  'maxNodeCount': 5,
  'nodeIdleTimeBeforeScaleDown': 'PT1200S'},
 'vmPriority': 'Dedicated',
 'vmSize': 'STANDARD_DS3_V2'}

Both training and testing instances are showing correctly. Now we can move to the training part.

### Training

In [14]:
from azureml.train import automl

In UI one was able to choose from veriaty of different metrics, all will be listed below. I think i will go down the easy and simlest root, and for trining ill choose **normalized_root_mean_squared_error**

In [15]:
automl.utilities.get_primary_metrics('regression')

['r2_score',
 'normalized_mean_absolute_error',
 'spearman_correlation',
 'normalized_root_mean_squared_error']

#### Create model
Again most of the code was coppied either from the LAB code, or from some tutorials. The exact parameters of functions can be found in https://docs.microsoft.com/en-gb/azure/machine-learning/

In [28]:
import logging
import os
from azureml.train.automl import AutoMLConfig

project_folder = './automl'
os.makedirs(project_folder, exist_ok=True)

automl_config = AutoMLConfig(compute_target=aml_remote_compute,
                             task='regression',
                             primary_metric='spearman_correlation',
                             experiment_timeout_minutes=30,                            
                             training_data=train_ds,
                             label_column_name='price',
                             n_cross_validations=5,
                             enable_early_stopping=True,
                             featurization='auto',
                             debug_log='automated_ml_errors.log',
                             verbosity=logging.INFO,
                             path=project_folder
                             )

AML model was created succesfully, will see further if it works correctly.

In [29]:
from azureml.core import Experiment
from datetime import datetime

now = datetime.now()
time_string = now.strftime("%m-%d-%Y-%H")
experiment_name = "regression-automl-1-{0}".format(time_string)
print(experiment_name)

experiment = Experiment(workspace=work_space, name=experiment_name)

import time
start_time = time.time()
            
run = experiment.submit(automl_config, show_output=True)

print('Manual run timing: --- %s seconds needed for running the whole Remote AutoML Experiment ---' % (time.time() - start_time))

regression-automl-1-01-26-2021-14
Running on remote.
No run_configuration provided, running on traininstance002 with default configuration
Running on remote compute: traininstance002
Parent Run ID: AutoML_4d23aadf-4acb-475a-a3d7-77afc18dfc85

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSE

### Results
One can explore the run details in the exporation tab

In [30]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [31]:
print(run.get_metrics())

{'experiment_status': ['DatasetEvaluation', 'FeaturesGeneration', 'DatasetFeaturization', 'DatasetFeaturizationCompleted', 'DatasetCrossValidationSplit', 'ModelSelection'], 'experiment_status_description': ['Gathering dataset statistics.', 'Generating features for the dataset.', 'Beginning to fit featurizers and featurize the dataset.', 'Completed fit featurizers and featurizing the dataset.', 'Generating individually featurized CV splits.', 'Beginning model selection.'], 'normalized_root_mean_squared_error': 0.017254612968118106, 'mean_absolute_error': 69030.8905441256, 'root_mean_squared_log_error': 0.17248953957998728, 'mean_absolute_percentage_error': 12.677510241514366, 'normalized_median_absolute_error': 0.005212822333520507, 'explained_variance': 0.87447501621208, 'median_absolute_error': 39732.131826093304, 'normalized_root_mean_squared_log_error': 0.03756097467110398, 'normalized_mean_absolute_error': 0.009056794875902075, 'root_mean_squared_error': 131514.6600429962, 'r2_scor

Other metrics:
- Explained variance 0.87712
- Mean absolute error 69129
- Mean absolute percentage error 13.071
- Median absolute error 40787
- Normalized mean absolute error 0.0090733
- Normalized median absolute error 0.0053534
- Normalized root mean squared error 0.016888
- Normalized root mean squared log error 0.038143
- R2 score 0.87707
- Root mean squared error 1.2867e+5
- Root mean squared log error 0.17372
- Spearman correlation 0.94596

Results seem to be very good, as almost no work was put into the model preparation, the whole thing took only 20min including the documentation reading, and tutorial run.

Also the training time was very short

In [32]:
import time
import datetime as dt

run_details = run.get_details()

end_time_utc_str = run_details['endTimeUtc'].split(".")[0]
start_time_utc_str = run_details['startTimeUtc'].split(".")[0]
timestamp_end = time.mktime(datetime.strptime(end_time_utc_str, "%Y-%m-%dT%H:%M:%S").timetuple())
timestamp_start = time.mktime(datetime.strptime(start_time_utc_str, "%Y-%m-%dT%H:%M:%S").timetuple())

parent_run_time = timestamp_end - timestamp_start
print('Run Timing: --- %s seconds needed for running the whole Remote AutoML Experiment ---' % (parent_run_time))

Run Timing: --- 2470.0 seconds needed for running the whole Remote AutoML Experiment ---


### Best Model testing
Best model is Voting Ensamble, so we will use it to test on test_ds

In [33]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
regression-automl-1-01-26-2021-14,AutoML_4d23aadf-4acb-475a-a3d7-77afc18dfc85,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [34]:
best_run, fitted_model = run.get_output()

ImportError: cannot import name 'UnhashableValueInColumn'

Unfortuantely I didnt manage to get the package running. I have tried to install 1.20 version in different combinations, but it didn't work. At the same time, all working models can be downloaded from the Models Tab, and later run via any ML framework, the model files will be attatched to the Git repo.